# Lab 4: Pretraga hiperparametara

## Zadatak: priprema i obrada Tips skupa podataka

1. **Učitavanje realnog skupa**: Učitajte taxis skup podataka upotrebom sns biblioteke: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
2. **Vektorizacija kategorickih podataka**: Identifikovati kategoričke kolone. Na odgovarajućim kolonama primjeniti LabelEncoder i OneHotEncoder
3. **Vremenski podaci** - Vektorizovati vremenske podatke na najbolji mogući način
4. **Imputacija**: Detektovati da li postoje NaN vrijednosti te ih rješiti odgovarajućom tehnikom. Detektovati kolone koje posjeduju visok stepen korelacije te izbrisati odgovarajuće kolone.
5. **Outlieri**: Definisati prag od 3 standardne devijacije za numeričke kolonu total_bill te izvršiti uklanjanje outlier-a.
6. **Skaliranje**: Primijeniti `StandardScaler` ili `MinMaxScaler` na total_bill koloni.
7. **Trening**: Podijeliti podatke na training, test i validacioni skup, te izvršiti pretragu hiperparametara. Koristiti decision tree model za regresiju. Ciljna kolona: tip.
8. **Evaluacija**: Kreirati finalni model sa najboljim parametrim i evaluirati metrike na testnom skupu podataka.

In [20]:
import seaborn as sns

df = sns.load_dataset('taxis')
print(f'Originalni oblik: {df.shape}')
df.head()

Originalni oblik: (6433, 14)


,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan
3,2019-03-10 01:23:59,2019-03-10 01:49:51,1,7.70,27.0,6.15,0.0,36.95,yellow,credit card,Hudson Sq,Yorkville West,Manhattan,Manhattan
4,2019-03-30 13:27:42,2019-03-30 13:37:14,3,2.16,9.0,1.10,0.0,13.40,yellow,credit card,Midtown East,Yorkville West,Manhattan,Manhattan


In [21]:
df.isnull().sum()

pickup              0
dropoff             0
passengers          0
distance            0
fare                0
tip                 0
tolls               0
total               0
color               0
payment            44
pickup_zone        26
dropoff_zone       45
pickup_borough     26
dropoff_borough    45
dtype: int64

In [23]:
from sklearn.impute import SimpleImputer

impute = ['payment','pickup_zone','dropoff_zone','pickup_borough','dropoff_borough']
imputer = SimpleImputer(strategy='most_frequent')
df[impute] = imputer.fit_transform(df[impute])
df.isnull().sum()

pickup             0
dropoff            0
passengers         0
distance           0
fare               0
tip                0
tolls              0
total              0
color              0
payment            0
pickup_zone        0
dropoff_zone       0
pickup_borough     0
dropoff_borough    0
dtype: int64

In [24]:
cat_cols = ['color','payment','pickup_zone','dropoff_zone','pickup_borough','dropoff_borough']
num_cols = ['passengers','distance','fare','tip','tolls','total']

In [25]:
from sklearn.preprocessing import LabelEncoder

df_encoded = df.copy()
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    le.fit(df[col])
    df_encoded[col] = le.fit_transform(df[col])
    label_encoders[col] = le

df_encoded.head()

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,1,1,99,176,2,2
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,1,0,174,182,2,2
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,1,1,1,190,2,2
3,2019-03-10 01:23:59,2019-03-10 01:49:51,1,7.70,27.0,6.15,0.0,36.95,1,1,86,202,2,2
4,2019-03-30 13:27:42,2019-03-30 13:37:14,3,2.16,9.0,1.10,0.0,13.40,1,1,116,202,2,2


In [26]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_data = ohe.fit_transform(df[cat_cols])
feature_names = ohe.get_feature_names_out(cat_cols)
df_ohe = pd.DataFrame(encoded_data, columns=feature_names, index=df.index)
df = df.drop(columns=cat_cols,axis=1).join(df_ohe)
df.head()

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color_green,color_yellow,...,dropoff_zone_Yorkville West,pickup_borough_Bronx,pickup_borough_Brooklyn,pickup_borough_Manhattan,pickup_borough_Queens,dropoff_borough_Bronx,dropoff_borough_Brooklyn,dropoff_borough_Manhattan,dropoff_borough_Queens,dropoff_borough_Staten Island
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2019-03-10 01:23:59,2019-03-10 01:49:51,1,7.70,27.0,6.15,0.0,36.95,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2019-03-30 13:27:42,2019-03-30 13:37:14,3,2.16,9.0,1.10,0.0,13.40,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [36]:
def process_time_duration(df):
    df = df.copy()
    df['duration'] = (df['dropoff'] - df['pickup']).dt.total_seconds() / 60
    df = df.drop(columns=['pickup','dropoff'], axis=1)
    return df

df = process_time_duration(df)
df.head()

,passengers,distance,fare,tip,tolls,total,color_green,color_yellow,payment_cash,payment_credit card,...,pickup_borough_Bronx,pickup_borough_Brooklyn,pickup_borough_Manhattan,pickup_borough_Queens,dropoff_borough_Bronx,dropoff_borough_Brooklyn,dropoff_borough_Manhattan,dropoff_borough_Queens,dropoff_borough_Staten Island,duration
0,1,1.60,7.0,2.15,0.0,-0.351768,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,6.250000
1,1,0.79,5.0,0.00,0.0,-0.888899,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,7.083333
2,1,1.37,7.5,2.36,0.0,-0.173705,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,7.400000
3,1,7.70,27.0,6.15,0.0,3.180056,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,25.866667
4,3,2.16,9.0,1.10,0.0,-0.285546,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,9.533333


In [37]:
import numpy as np
_total = df['total'].copy()
mask = np.abs(_total - _total.mean()) < 3*_total.std()
df = df[mask]
df.head()

,passengers,distance,fare,tip,tolls,total,color_green,color_yellow,payment_cash,payment_credit card,...,pickup_borough_Bronx,pickup_borough_Brooklyn,pickup_borough_Manhattan,pickup_borough_Queens,dropoff_borough_Bronx,dropoff_borough_Brooklyn,dropoff_borough_Manhattan,dropoff_borough_Queens,dropoff_borough_Staten Island,duration
0,1,1.60,7.0,2.15,0.0,-0.351768,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,6.250000
1,1,0.79,5.0,0.00,0.0,-0.888899,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,7.083333
2,1,1.37,7.5,2.36,0.0,-0.173705,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,7.400000
4,3,2.16,9.0,1.10,0.0,-0.285546,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,9.533333
5,1,0.49,7.5,2.16,0.0,-0.350296,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,10.133333


In [42]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scale = ['total','distance','fare','tolls']
df[scale] = scaler.fit_transform(df[scale])
df.head()

,passengers,distance,fare,tip,tolls,total,color_green,color_yellow,payment_cash,payment_credit card,...,pickup_borough_Bronx,pickup_borough_Brooklyn,pickup_borough_Manhattan,pickup_borough_Queens,dropoff_borough_Bronx,dropoff_borough_Brooklyn,dropoff_borough_Manhattan,dropoff_borough_Queens,dropoff_borough_Staten Island,duration
0,1,-0.272732,-0.580389,2.15,-0.066312,-0.327300,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,6.250000
1,1,-0.735825,-0.938224,0.00,-0.066312,-0.908922,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,7.083333
2,1,-0.404227,-0.490930,2.36,-0.066312,-0.134488,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,7.400000
4,3,0.047432,-0.222554,1.10,-0.066312,-0.255593,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,9.533333
5,1,-0.907341,-0.490930,2.16,-0.066312,-0.325707,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,10.133333


In [45]:
from sklearn.model_selection import train_test_split

X = df.drop('tip',axis=1).to_numpy()
Y = df['tip'].to_numpy()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [48]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

model = AdaBoostRegressor(DecisionTreeRegressor())

model.fit(X_train, Y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor())

In [49]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

Y_pred = model.predict(X_test)

mae = mean_absolute_error(Y_pred, Y_test)
mse = mean_squared_error(Y_pred, Y_test)
print(f"MAE: {mae}")
print(f"MSE: {mse}")

MAE: 0.1322657580919932
MSE: 0.12312333901192503
